<a href="https://colab.research.google.com/github/NiyaziOnurYantira/FineTuning/blob/main/llama-8b-tr-en-medicalBot-INFERENCE/Inference_tr_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:
MODEL_NAME = "OnurYantira/llama3-8b-turkish-english-medical-merged"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [5]:
system_prompt = "You are a helpful medical assistant. Provide concise, accurate, and professional answers to the user's health-related questions."
user_message = "A patient is experiencing fatigue, brittle nails, and pale skin. What could be the possible cause and what tests would you recommend?"

In [6]:
prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    f"{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    f"{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
)

In [7]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

In [9]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].strip()

In [10]:
print("Model Yanıtı:")
print(response)

Model Yanıtı:
system
You are a helpful medical assistant. Provide concise, accurate, and professional answers to the user's health-related questions.user
A patient is experiencing fatigue, brittle nails, and pale skin. What could be the possible cause and what tests would you recommend?assistant
The symptoms described by the patient - fatigue, brittle nails, and pale skin - may be indicative of Iron Deficiency Anemia. Here are some potential causes and recommended diagnostic tests: Possible Causes: 1. Poor iron absorption in the gut 2. Chronic blood loss due to menstruation or gastrointestinal issues 3. Inadequate dietary intake of iron-rich foods Recommended Tests: 1. Complete Blood Count (CBC) to confirm anemia 2. Ferritin test to measure iron stores in the body 3. Total Iron Binding Capacity (TIBC) to assess the amount of transferrin available for iron transport 4. Transferrin saturation test to determine how much iron is bound to transferrin 5. Serum Iron test to evaluate the level

In [11]:
system_prompt = "Sen Türkçe konuşan yardımcı bir medikal asistansın. Kullanıcının sağlıkla ilgili sorularına açık, doğru ve profesyonel yanıtlar ver."
user_message = "Bir hastada halsizlik, solgunluk ve tırnaklarda kırılma şikayetleri var. Bu durum neye işaret edebilir ve hangi testler yapılmalıdır?"

#  LLaMA 3.1 chat formatına uygun prompt
prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    f"{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    f"{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
)

#  Tokenize ve modele gönder
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
model.to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

#  Yanıtı ayıkla ve göster
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = generated_text.split("<|start_header_id|>assistant<|end_header_id|>\n")[-1].strip()

print(" Model Yanıtı (Türkçe):")
print(response)

 Model Yanıtı (Türkçe):
system
Sen Türkçe konuşan yardımcı bir medikal asistansın. Kullanıcının sağlıkla ilgili sorularına açık, doğru ve profesyonel yanıtlar ver.user
Bir hastada halsizlik, solgunluk ve tırnaklarda kırılma şikayetleri var. Bu durum neye işaret edebilir ve hangi testler yapılmalıdır?assistant
Hastanın yaşadığı belirtiler anemiye (kansızlık) işaret edebilir. Anemide tırnakların kırılması, halsizlik, nefes darlığı, bulantı ve baş ağrısı gibi belirtiler olabilir. Anemi teşhisi konulmak için kan testsi yapılması gereklidir.
